<a href="https://colab.research.google.com/github/TusharK54/dotfiles/blob/master/4740_FA20_p1_tak62.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1: Language Modeling and Fake News Classification

Name: Tushar Khan

Netid: tak62

**After you make your own copy, please rename this notebook by clicking on it's name in the upper left corner.** It should be named: 4740_FA20_p1_netid1_netid2

Don't forget to share your newly copied notebook with your partner!

**Reminder: both of you can't work in this notebook at the same time from different computers/browser windows because of sync issues. We even suggest to close the tab with this notebook when you are not working on it so your partner doesn't get sync issues.**

## Introduction
In this project we will build an **n-gram-based language model** for fake news classification. We will also investigate a feature-based **Naive Bayes model**. The task we are faced with is to **decide whether a news article is fake or real**. While some fake articles are so absurd and nonsensical that one can clearly guess they are fake, most fake news is actually quite hard to detect. [Various studies](https://pacscenter.stanford.edu/research/program-on-democracy-and-the-internet/deception-detection-accuracy-for-fake-news-headlines-on-social-media/) have shown that most people can have an error rate up to 50% depending on the theme of the article. 

To help us approach this problem, we will use NLP techniques covered thus far to frame this as a (supervised) binary classification task, where each article will have a label $y \in \{0,1\}$, where *0 indicates a fake article* and *1 indicates a real one*. You will train and validate your two different models and then run them on a test data set with hidden $y$ labels. You will then submit the results on the test data set to Kaggle to participate in our class-wide competition!

The project is divided into six parts:
1. Dataset loading and preprocessing
2. Unsmoothed n-gram language model (LM): build the unsmoothed n-gram language model using our Fake News corpus. 
3. Smoothed n-gram language model: build a smoothed version of the model from part 2.
4. Perplexity: compute perplexity for both the unsmoothed and smoothed model
5. Putting everything together and submitting the first model to Kaggle
6. Naive Bayes: build a feature-based Naive Bayes model to perform the same classification task. Compare the LM with Naive Bayes and identify the pros and cons of each.

**Logistics:** You should work in **groups of 2 students**. Students in the same group will get the same grade. Thus, you should make sure that everyone in your group contributes to the project. 
- **Remember to form groups on BOTH CMS and Gradescope** or not all group members will receive grades. You can use the Teammate Search option on Piazza to find a partner for this project.

**Advice:** Please complete the written parts of this notebook in a clear and informative way. This is where you get to show us that you understand not only what you are doing but also why and how you are doing it. So be clear, organized and concise; avoid vagueness and excess verbiage. Spend time
doing error analysis for the models. This is how you understand the advantages and drawbacks of the systems you build. 
- It's also useful to think about how the theory of n-grams/Naive Bayes bridges with the real world application we are building. Think about what you expect from these models based on your current understanding, and then see if your expectation aligns with empirical results that you'll get. 

## General Guidelines
In this project, we provide a few code snippets or starter points in case you need them. You DO NOT need to follow the structure. 

If you think you have a better idea, go for it. You can ADD, MODIFY, or DELETE any code snippets given to you.

**Let's do this** 🚀

### Dataset

You are given a **News Corpus** on CMS, which consists of roughly the same amount of real and fake news articles.

Real news example:
```
The OpenAI technology, known as GPT-2, is designed to predict the next word given all the previous words it is shown within some text. The language-based model has been trained on a dataset of 8 million web pages.
```

Fake news example:
```
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English.
```

In the dataset folder you should find 4 files, training and validation splits for both real and fake news.

The project will proceed generally as follows in terms of code development:
1. Write code to train unsmoothed unigram and bigram language models for an arbitrary corpus
2. Implement smoothing and unknown word handling. 
3. Implement the Perplexity calculation. 
4. Using 1, 2 and 3, together with the provided training and validation sets, develop a language-model-based approach for Fake News Classification.
5. Apply your best language-model-based news classifier (from 4) to the
provided test set. Submit the results to the online Kaggle competition. 
6. Use any existing implementation of Naive Bayes (and the provided training and validation sets) to create an additional Naive Bayes fake news classifier. Apply your best NB classifier to the provided test set. Submit the results to the separate Kaggle competition (for NB classifiers). 

We will progress towards these tasks throughout this notebook.

# Part 1: Preprocessing the Dataset
In this part, you are going to do a few things:
* Connect to the google drive where the data set is stored
* Load and read files
* Preprocess the text

------
**Please upload the dataset to each partner's individual Google Drive now.** We suggest using the same folder structure within Google Drive because the notebook is shared among you, so the code to load the data would have to be changed every time if folder structures are different. One folder structure might be: Google Drive/CS 4740/Project 1/Dataset/ or whatever works for you. See our code below for an example of how we load the data from Google Drive.

## 1.1 Connect to google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## 1.2 Load and read files
First, let's install [NLTK](https://www.nltk.org/), a very widely package for NLP preprocessing (and other tasks) for Python.

In [2]:
!pip install -U nltk

Requirement already up-to-date: nltk in /usr/local/lib/python3.6/dist-packages (3.5)


Then we read and load data.

In [3]:
import os
import io
from nltk import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

root_path = os.path.join(os.getcwd(), "drive", "My Drive/Colab Notebooks/Project 1") # replace based on your Google drive organization
dataset_path = os.path.join(root_path, "dataset") # same here

with io.open(os.path.join(dataset_path, "trueDataTrain.txt"), encoding='utf8') as real_file:
  real_news = real_file.read()
with io.open(os.path.join(dataset_path, "trueDataValidation.txt"), encoding='utf8') as real_file:
  real_news_validation = real_file.read()
with io.open(os.path.join(dataset_path, "fakeDataTrain.txt"), encoding='utf8') as fake_file:
  fake_news = fake_file.read()
with io.open(os.path.join(dataset_path, "fakeDataValidation.txt"), encoding='utf8') as real_file:
  fake_news_validation = real_file.read()

tokenized_real_news_training = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(real_news)]
tokenized_fake_news_training = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(fake_news)]
tokenized_real_news_validation = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(real_news_validation)]
tokenized_fake_news_validation = [list(map(str.lower, word_tokenize(sent))) for sent in sent_tokenize(fake_news_validation)]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Sanity checks for our real and fake training sets

In [4]:
tokenized_real_news_training[0]

['london',
 '(',
 'reuters',
 ')',
 '-',
 'britain',
 'is',
 'seeking',
 'to',
 'build',
 'on',
 'the',
 'recent',
 'momentum',
 'in',
 'the',
 'brexit',
 'divorce',
 'talks',
 'with',
 'the',
 'european',
 'union',
 'before',
 'a',
 'summit',
 'next',
 'month',
 ',',
 'a',
 'spokeswoman',
 'for',
 'britain',
 's',
 'department',
 'for',
 'exiting',
 'the',
 'european',
 'union',
 'said',
 'on',
 'tuesday',
 '.']

In [5]:
tokenized_fake_news_training[0]

['rick',
 'santorum',
 'had',
 'the',
 'impossible',
 'job',
 'of',
 'defending',
 'donald',
 'trump',
 'during',
 'friday',
 's',
 'real',
 'time',
 'and',
 'he',
 'found',
 'himself',
 'humiliatingly',
 'outnumbered.bill',
 'maher',
 'began',
 'by',
 'expressing',
 'astonishment',
 'at',
 'how',
 'the',
 'republican',
 'nominee',
 'has',
 'been',
 'acting',
 'throughout',
 'the',
 'campaign',
 '.']

## 1.3 Data Preprocessing & Preparation

There's a well-known parable in machine learning that 80% of the work is all about data preparation, 10% is supporting infrastructure and 10% is actual modeling. If your "raw" dataset is not preprocessed and prepared in a way to maximize its value, then your model will be more like this: https://xkcd.com/1838/. For this project, modeling is the star of the show for learning purposes, but we still want you to pay attention to the preprocessing stage.

*We've already tokenized and lowercased* the raw data for you. Here are a few extra things you might want to do:

- Think about edge cases. For example, you don't want to accidentally append a period to the last word of a sentence. 
- Watch out for apostrophes and other tricky things like quotations, they cause lots of edge cases. For example, "they're" can be all one token, or two tokens ("they", "'re") or even three tokens ("they", " ' ", "re"). 

Why did we lowercase all tokens? Because the computer will otherwise consider "The" and "the" as two separate words and this will cause problems.

Note that you may use existing
tools just for the purpose of preprocessing. 

Advice: don't get bugged down in the dozens of preprocessing packages and suggestions that you can find on Towards Data Science or Stack Overflow. Start with this [NLTK tutorial](https://lost-contact.mit.edu/afs/cs.pitt.edu/projects/nltk/docs/tutorial/introduction/nochunks.html#:~:text=The%20Natural%20Language%20Toolkit%20(NLTK,tokenization%2C%20tagging%2C%20and%20parsing.) and that should be plenty.

In [6]:
# TODO

**Q1.1: Show some observations or statistics from the dataset** (should be quantitative – i.e. most frequent words, most frequent bigram, etc.)

I was curious about the distribution of word lengths in the texts, and if it was different between the real and fake news datasets. 

I found that more than half of the words in both datasets are between 1 and 4 (inclusive) characters. This accounted for 55% of the words in the real news training set and 57% of the words in the fake news training set.

I also found that for the most part, the distributions did not significantly differ between the datasets. However, the range of word lengths is much larger in the fake news dataset. The longest word in the real news dataset was 98 characters, while the longest word in the fake news dataset was 1057 characters.

The functions I used to compute these values is below.

In [10]:
def get_word_lengths(lst):
  length_freq = {}

  for sentence in lst:
    for word in sentence:
      length = len(word)
      if length in length_freq:
        length_freq[length] += 1
      else:
        length_freq[length] = 1

  count = sum(length_freq.values())
  length_freq = {k: v/count for k, v in length_freq.items()}

  return dict(sorted(length_freq.items()))

In [11]:
get_word_lengths(tokenized_real_news_training)

{1: 0.14656622098172942,
 2: 0.13398942117196005,
 3: 0.15205353720775422,
 4: 0.131475069823566,
 5: 0.10103985001610725,
 6: 0.08408826530082052,
 7: 0.08273580620926958,
 8: 0.05876029779162533,
 9: 0.04751486788079288,
 10: 0.028899682072780333,
 11: 0.014518380912385713,
 12: 0.008406641200702484,
 13: 0.004642525959573546,
 14: 0.0030013894415887436,
 15: 0.0011926091243461587,
 16: 0.000444095606784974,
 17: 0.0002984579215592065,
 18: 0.00014915255066143656,
 19: 7.167269084038293e-05,
 20: 5.0736318462701776e-05,
 21: 6.112809453337564e-05,
 22: 1.0850236779674175e-05,
 23: 7.641011816671954e-06,
 24: 5.04306779900349e-06,
 25: 3.514865435669099e-06,
 26: 1.833842836001269e-06,
 27: 1.0697416543340736e-06,
 28: 2.9035844903353428e-06,
 29: 3.667685672002538e-06,
 30: 6.112809453337564e-07,
 31: 1.528202363334391e-07,
 32: 1.528202363334391e-07,
 33: 1.528202363334391e-07,
 38: 1.528202363334391e-07,
 39: 3.056404726668782e-07,
 42: 1.528202363334391e-07,
 52: 1.528202363334391

In [12]:
get_word_lengths(tokenized_fake_news_training)

{1: 0.13647876498532782,
 2: 0.14134659141095168,
 3: 0.1686364300815557,
 4: 0.14521311461861375,
 5: 0.10495256400256821,
 6: 0.07789878185536295,
 7: 0.07426327895424377,
 8: 0.05097064686202367,
 9: 0.0385029109333427,
 10: 0.0249287569015541,
 11: 0.013679656318231053,
 12: 0.008533546627878327,
 13: 0.0053886229020282785,
 14: 0.0032338968903039655,
 15: 0.002127091385899729,
 16: 0.0009843866946766414,
 17: 0.0009267930352478361,
 18: 0.00042872410157765304,
 19: 0.00027311791410745067,
 20: 0.0001773006600801559,
 21: 0.00010821409551869676,
 22: 9.46550501374758e-05,
 23: 9.194324106123162e-05,
 24: 2.6085020638158407e-05,
 25: 2.156533884441809e-05,
 26: 0.0005818767475026821,
 27: 2.763462582458366e-05,
 28: 8.006293463197135e-06,
 29: 5.294484386952944e-06,
 30: 7.877159697661697e-06,
 31: 4.39054802820488e-06,
 32: 3.744879200527692e-06,
 33: 2.0661402485670025e-06,
 34: 1.4204714208898143e-06,
 35: 1.756219211281952e-05,
 36: 1.549605186425252e-06,
 37: 4.39054802820488e-

**Please answer the following question**:

**Q1.2: What did you do in your preprocessing part?**

TODO

# Part 2: Compute Unsmoothed Language Models.

To start, you will write a program that computes unsmoothed unigram and bigram probabilities. You should consider real and fake news as separate corpora and
generate a separate language model for each set of news.
We have already loaded the data and (partially) preprocessed it and you probably did some of your own preprocessing. 

Note that you were allowed to use existing
tools for the purpose of preprocessing, but you must write the code for gathering n-gram counts and computing n-gram probabilities yourself. 

For example, consider the
simple corpus consisting of the sole sentence:


> the students liked the assignment

Part of what your program would compute for a unigram and bigram model, for example,
would be the following:


> $P("the") = 0.4; P("liked") = 0.2; P("the"|"liked") = 1.0; P("students"|"the") = 0.5$

Remember to add a symbol to mark the beginning of sentence. See Sept. 7th lecture, p25-28 for an example.




In [101]:
print_text=tokenized_fake_news_training[3]

In [78]:
# Returns a dictionary that maps a unigram : str to its number of occurrences
# Requires `text` is a list of tokens : str
def unigram_counts(text):
  counts = {}

  for t in text:
    counts[t] = 1 if t not in counts else counts[t] + 1

  return counts

In [79]:
# Returns a dictionary that maps a bigram : (str, str) to its number of occurrences
# Requires `text` is a list of tokens : str
def bigram_counts(text):
  words = set(text)
  counts = {}       ## we don't store counts that are 0

  for i in range(1, len(text)):
    b = (text[i-1], text[i])
    counts[b] = 1 if b not in counts else counts[b] + 1

  return counts

In [105]:
unigram_counts(print_text)

{'.': 1,
 'he': 1,
 'it': 1,
 'like': 1,
 'lose': 1,
 'not': 1,
 's': 2,
 'to': 1,
 'trying': 1}

In [104]:
bigram_counts(print_text)

{('he', 's'): 1,
 ('it', 's'): 1,
 ('like', 'he'): 1,
 ('lose', '.'): 1,
 ('not', 'like'): 1,
 ('s', 'not'): 1,
 ('s', 'trying'): 1,
 ('to', 'lose'): 1,
 ('trying', 'to'): 1}

## 2.1 Unsmoothed Uni-gram Model

> Indented block



In [93]:
# Returns a dictionary that maps a unigram : str to its (prior) probability
# Requires `text` is a list of tokens : str
def unigram_model(text):
  counts_dict = unigram_counts(text) # dictionary maps tokens to counts
  total_count = sum(counts_dict.values())
  probabilities = {t : c/total_count for t, c in counts_dict.items()}
  return probabilities

In [102]:
unigram_model(print_text)

{'.': 0.1,
 'he': 0.1,
 'it': 0.1,
 'like': 0.1,
 'lose': 0.1,
 'not': 0.1,
 's': 0.2,
 'to': 0.1,
 'trying': 0.1}

## 2.2 Unsmoothed Bi-gram Model

In [94]:
# Returns a dictionary that maps a bigram : (str, str) to its (prior) probability
# Requires `text` is a list of tokens : str
def bigram_model(text):
  # dictionaries mapping n-grams to counts
  unigram_count_dict = unigram_counts(text)
  bigram_counts_dict = bigram_counts(text)

  probabilities = { (t1, t2) : c/unigram_count_dict[t1] for (t1, t2), c in bigram_counts_dict.items() }
  return probabilities

In [103]:
bigram_model(print_text)

{('he', 's'): 1.0,
 ('it', 's'): 1.0,
 ('like', 'he'): 1.0,
 ('lose', '.'): 1.0,
 ('not', 'like'): 1.0,
 ('s', 'not'): 0.5,
 ('s', 'trying'): 0.5,
 ('to', 'lose'): 1.0,
 ('trying', 'to'): 1.0}

**Q2: What data structure are you using to store probabilities for unigrams and bigrams? Why did you select this data structure?**

I used a dictionary (i.e. hash table) because we will need to index the count and probability of a given unigram or bigram, and dictionaries have a constant-time indexing operation.


# Part 3: Smoothed Language Model
In this part, you will need to implement **at least one** smoothing method and **at least one** method to handle unknown words in the test data. You can choose any method(s) that you want for each. You should make clear
**what method(s)** were selected and **why**, providing a description for any non-standard approach (e.g., an approach that was not covered in class or in the readings). 

You should use the
provided validation sets to experiment with different smoothing/unknown word handling
methods if you wish to see which one is more effective for this task. (We will cover this in Part 4).

## 3.1 Unknown Words Handling

**Q3.1: How are you going to handle unknown words? What parameters might be needed? Do you need a method to determine the value?**

We first sort the tokens by their frequency. We then take the bottom `alpha`-percentile of tokens and replace them with the symbol for unknown words `UNK`. This `UNK` symbol is treated as any other token in the n-gram probability computations.

This is equivalent to implicitly creating a closed vocabulary of the `(1-alpha)` percent most frequent tokens. 

We will need to choose `alpha` using a validation set.



In [21]:
UNK = '<UNK>' # marks an unknown word

In [116]:
# Returns the text with the bottom `alpha`-percentile of tokens by frequency
#   replaced with the unknown word symbol UNK
# Requires `text` is a list of tokens : str
def generate_unk_text(text, alpha):
  # compute frequency of each token
  token_freq = unigram_counts(text)
  # sort tokens in order of ascending frequency
  sorted_tokens = sorted(token_freq, key=token_freq.get)
  # get the bottom `alpha`-percentile of tokens by frequency
  alpha_tokens = set(sorted_tokens[:int(len(sorted_tokens)*alpha)])
  # map original text with UNK-replacement funcion
  unk_text = list(map(lambda t: UNK if t in alpha_tokens else t, text))
  return unk_text

In [117]:
generate_unk_text(print_text, 0.3)

['<UNK>', 's', '<UNK>', 'like', 'he', 's', 'trying', 'to', 'lose', '.']

## 3.2 Smoothing

In this part of project, we are going to compute the probabilities for unigram and bigram models after smoothing.
There are several smoothing methods you can start with:
* add-k
* Kneser-Ney
* Good-Turing
* ...

You need to compute for both unigram and bigram models.

Below is a starter point using add-k smoothing. As always, you DO NOT need to follow it; you DO NOT need to use add-k smoothing if you do not want to. You can pick ANY smoothing method you want.

In [24]:
"""
Function [add_k_unigram] applies add-k smoothing to a unigram model
dic: a dictionary of your unigrams. key: words, val: occurence
k: parameter k for smoothing
Return: a dictionary of results after smoothing
"""
def add_k_unigram(dic, k):
  total = sum(dic.values())
  unique = len(dic)
  return {w: (v+k)/(total + k*unique) for w, v in dic.items()}

def 

In [25]:
lst=tokenized_fake_news_training[3]
dic=unigram_counts(lst)
add_k_unigram(dic, 1)

{'.': 0.10526315789473684,
 'he': 0.10526315789473684,
 'it': 0.10526315789473684,
 'like': 0.10526315789473684,
 'lose': 0.10526315789473684,
 'not': 0.10526315789473684,
 's': 0.15789473684210525,
 'to': 0.10526315789473684,
 'trying': 0.10526315789473684}

In [67]:
"""
Function [add_k_bigram] applies add-k smoothing to a bigram model
uni_dic: a dictionary of your unigrams.
bi_dic: a dictionary of your bigrams.
k: parameter k for smoothing
Return: a function that takes a bigram and returns a probability
"""
def add_k_bigram(uni_dic, bi_dic, k):
  unique = len(uni_dic)
  return {(w0, w1): (v+k)/(uni_dic[w0] + k*unique) for (w0, w1), v in bi_dic.items()}

# Returns k-smoothed probability of `bigram` given MLE counts
# Note that `bigram` is a pair (w0, w1) of tokens 
def get_k_bigram(uni_counts, bi_counts, k, bigram):
  unsmoothed = bi_counts[bigram]
  



In [27]:
lst=tokenized_fake_news_training[3]
uni_dic=unigram_counts(lst)
bi_dic=bigram_counts(lst)
add_k_bigram(uni_dic, bi_dic, 0.5)

{('.', '.'): 0.09090909090909091,
 ('.', 'he'): 0.09090909090909091,
 ('.', 'it'): 0.09090909090909091,
 ('.', 'like'): 0.09090909090909091,
 ('.', 'lose'): 0.09090909090909091,
 ('.', 'not'): 0.09090909090909091,
 ('.', 's'): 0.09090909090909091,
 ('.', 'to'): 0.09090909090909091,
 ('.', 'trying'): 0.09090909090909091,
 ('he', '.'): 0.09090909090909091,
 ('he', 'he'): 0.09090909090909091,
 ('he', 'it'): 0.09090909090909091,
 ('he', 'like'): 0.09090909090909091,
 ('he', 'lose'): 0.09090909090909091,
 ('he', 'not'): 0.09090909090909091,
 ('he', 's'): 0.2727272727272727,
 ('he', 'to'): 0.09090909090909091,
 ('he', 'trying'): 0.09090909090909091,
 ('it', '.'): 0.09090909090909091,
 ('it', 'he'): 0.09090909090909091,
 ('it', 'it'): 0.09090909090909091,
 ('it', 'like'): 0.09090909090909091,
 ('it', 'lose'): 0.09090909090909091,
 ('it', 'not'): 0.09090909090909091,
 ('it', 's'): 0.2727272727272727,
 ('it', 'to'): 0.09090909090909091,
 ('it', 'trying'): 0.09090909090909091,
 ('like', '.'): 0.

**Please answer the following question:**

**Q3.2: Which smoothing method did you choose? Are there any parameters, if so how are you planning to pick the value? If you choose to implement more than 1 method (not a requirement), please state each of them. Providing a description for any non-standard approach, e.g., an approach that was not covered in class or in the readings**

I chose to use add-k smoothing, which only has a single parameter, that being k. I will pick the value of k by optimizing on a validation set.

# Part 4: Perplexity
At this point, we have developed several language models: unigram vs bigram, unsmoothed vs smoothed. We now want to compare all the models. 

Implement code to compute the perplexity of a **“development set.”** (“Development set”
is just another way to refer to the validation set—part of a dataset that is distinct from
the training portion and the test portion.) Compute and report the perplexity of each
of the language models (one trained on true news and fake news) on
the development corpora. Compute perplexity as follows:
\begin{align*}
PP &= \left(\prod_i^N\frac{1}{P\left(W_i\mid W_{i-1}, ...W_{i-n+1}\right)}\right)^{\frac{1}{N}}\\
&=\exp \frac{1}{N}\sum_{i}^N-\log P\left(W_i\mid W_{i-1}, ...W_{i-n+1}\right)
\end{align*}
where $N$ is the total number of tokens in the test corpus and $P\left(W_i\mid W_{i-1}, ...W_{i-n+1}\right)$
is the n-gram probability of your model. Under the second definition above, perplexity
is a function of the average (per-word) log probability: use this to avoid numerical
computation errors.

Please complete the following tasks and report what you have observed. Remember, lower perplexity means better model.

## Task 1: Compute perplexity for smoothed unigram and smoothed bigram. 
*Note: If you choose more than one smoothing method, pick one of them to compute. If you need to try different values of parameters, you can try them out here.*


In [51]:
START = '<START>' # marks the beginning of a sentence
END   = '<END>'   # marks the end of a sentence

In [52]:
"""
Function [tokenize_text] processes a list of tokenized sentences
text: a list of sentences which are tokenized
Return: a list of tokens with special tokens inserted where appropriate
"""
def tokenize_text(text):
  new_text = []
  for i in range(len(text)):
    new_text += [START] + text[i] + [END]
  return new_text

In [59]:
processed_real_news_training = tokenize_text(tokenized_real_news_training)
processed_fake_news_training = tokenize_text(tokenized_fake_news_training)
processed_real_news_validation = tokenize_text(tokenized_real_news_validation)
processed_fake_news_validation = tokenize_text(tokenized_fake_news_validation)

In [73]:
import math

"""
train: the (processed) training set
dev: the (processed) development set
alpha: the percentage of the most infrequent word types to replace with UNK
k: the value of k for add-k smoothing
Return: the perplexity of a unigram model on the development set
"""
def smooth_unigram_perplexity(train, dev, alpha=0.1, k=0.5):
  unigram = train_unigram(train, alpha, k)

  sum_term = 0
  for w in dev:
    try:
      probability = -1 * math.log(train_unigram[w])
    except KeyError:
      probability = -1 * math.log(train_unigram[UNK])
    sum_term += probability

  return math.exp(sum_term / len(dev))

In [74]:
real_pp_unigram = smooth_unigram_perplexity(processed_real_news_training, processed_real_news_validation)
fake_pp_unigram = smooth_unigram_perplexity(processed_fake_news_training, processed_fake_news_validation)

print("Unigram PP")
print("Real news model:", real_pp_unigram)
print("Fake news model:", fake_pp_unigram)

Unigram PP
Real news model: 873.0566349642545
Fake news model: 1026.0685770305938


In [75]:
import math

"""
train: the (processed) training set
dev: the (processed) development set
alpha: the percentage of the most infrequent word types to replace with UNK
k: the value of k for add-k smoothing
Return: the perplexity of a unigram model on the development set
"""
def smooth_bigram_perplexity(train, dev, alpha=0.1, k=0.5):
  bigram = train_bigram(train, alpha, k)

  sum_term = 0
  for i in range(1, len(dev)):
    w0, w1 = text[i-1], text[i]
    b = (w0, w1)
    
    try:
      probability = -1 * math.log(train_bigram[b])
    except KeyError:
      probability = -1 * math.log(train_bigram[(w0, UNK)])
    sum_term += probability

  return math.exp(sum_term / len(dev))

In [77]:
real_pp_bigram = smooth_bigram_perplexity(processed_real_news_training, processed_real_news_validation)
fake_pp_bigram = smooth_bigram_perplexity(processed_fake_news_training, processed_fake_news_validation)

print("Bigram PP")
print("Real news model:", real_pp_bigram)
print("Fake news model:", fake_pp_bigram)

KeyboardInterrupt: ignored

**Q4.1: Why do we need to compute perplexity after smoothing?**

N-gram models are subject to data sparsity because most corpa aren’t
large enough to contain all possible N-grams.

Any N-gram that is not in the training set is computed to have a probability of 0.

If the probability of any word in the test set is computed to be 0, the probability of the entire test set is also computed to be 0. Since we cannot take the inverse, we cannot compute perplexity, and therefore cannot evaluate the model at all.

Therefore we need to introduce smoothing before computing perplexity.

**Q4.2: Did you choose any values for parameters?**

I had to choose values for alpha and k. I chose alpha to be 0.1 since there wasn't much improvement above that value, and intuitively I don't want to remove to many infrequent words from the dataset. I chose k to be 0.5 although this value seemed to affect the total perplexity much less.

## Task 2: Compute perplexity for other smoothing methods (optional). 
*Note: If you only pick one smoothing method, you can omit this task. If you need to try different values of parameters, you can try them out here.*

In [31]:
# TODO: compute perplexity for your rest of smoothing method.

**Q4.3: If your smoothing method needs to pick a parameter, what is the value of your parameter?**

Your answer:

**Q4.4: Which smoothing method is the best among your choices?**

Your answer:

# Part 5: Putting Everything Together and Submitting to Kaggle
Combining all the previous parts together, we have developed a bunch of language models. Before we proceed to the next step, let's check a few things (no need to answer):
* Did you train your model only on training set?
* Did you validate your model only on validation/development set?
* Did you determine all your parameters?

Finally, please answer:

**Q5: What is your choice of language model, and why?**

I chose to use the smoothed bigram language model since that had the lowest PP.



In [32]:
# TODO: anything that helps you answer/check the above points.

## Part 5.1: First Model Submission to Kaggle

Now we need to apply our model to testing data. What you need to do:
* Takes the test data as input, and generates an output of your prediction based on your chosen language model
* Your output file should be ONLY your predictions
* Submit to Kaggle

You should use your trained model to predict labels for all the news in `TestData.txt`. Output your predictions to a **csv** file and submit it to kaggle. Each line should contain the id of the test news and its corresponding prediction (in total 4489 lines). In other words, your output should look like (**including the header**):
```
Id,Prediction
0,0
1,0
2,1
3,0
...
4488,1
```
Note that you should add the header `Id,Prediction` and there is no space in the output. The Id starts from 0 (not 1).

Use this kaggle [link](https://www.kaggle.com/t/a032d54898b24b4bb2f29e7165ac5fdf) to submit your output. Your team name should be the concatenation of your netids, **exactly in the same order as this notebook is named**. For example, if notebook is 4740_FA20_p1_jb123_cj456, then Kaggle group should be jb123_cj456.


In [33]:
# TODO: Add code to generate the Kaggle output file and submit the output file to Kaggle

# Part 6: Naive Bayes

The Naive Bayes classification method is based on Bayes Rule. Suppose we have a news article *d* and its label *c* (either 0 or 1).
\begin{align*}
P(c|d)=\frac{P(d|c)P(c)}{P(d)}
\end{align*}
Likelihood: $P(d|c)$. In real/fake corpus, how likely *d* would appear.

Prior: $P(c)$. The probability of real/fake news in general.

Posterior: $P(c|d)$. Given *d*, how likely is it that it is real/fake.

Goal: $\underset{c\in \{0,1\}}{\operatorname{argmax}} P(c|d)$, which is equivalent to $\underset{c\in \{0,1\}}{\operatorname{argmax}} P(d|c)P(c)$.

The equivalence holds because $P(d)$ is the same for any $c$. Thus the denominator can be dropped.

Denote $d=\{x_1, x_2, ..., x_n\}$ where $x_i$'s are words in the news *d* (sometimes called features). Unlike n-gram language modelling, we make the multinomial Naive Bayes independence assumption here, where we assume positions of words do not matter. Formally, 
\begin{align*}
&\underset{c\in \{0,1\}}{\operatorname{argmax}} P(d|c)P(c)\\
=&\underset{c\in \{0,1\}}{\operatorname{argmax}} P(x_1, ..., x_n|c)P(c)\\
=&\underset{c\in \{0,1\}}{\operatorname{argmax}} P(x_1|c)P(x_2|c)...P(x_n|c)
\end{align*}

Now we only need to collect the occurences of each word for the classification. This is often called a **bag of words** feature. 

For instance, in the sentence `All for one and one for all .`, the bag of words feature would be `{"all": 2, "for": 2, "one": 2, "and": 1, ".": 1}`. Essentially, the bag of words feature is a dictionary which maps the word to its occurences. We can see that the order is not considered here.

Now, your goal is to implement the Multinomial Naive Bayes. You can use existing codes or Python packages, and adapt them to our news classification task.

You might find the following packages/functions useful:

* nltk.word_tokenize(), nltk.word_tokenize()
* nltk.classify.naivebayes()
* sklearn.feature_extraction.text
* sklearn.naive_bayes.MultinomialNB()

**Please answer the following question(s).**

**Q6: Comparing Multinomial Naive Bayes with the unigram language model, which one do you expect to perform better? Why?**

A: 

## 6.1 Implementation

In [34]:
# TODO: Naive Bayes implementation 

## 6.2 Putting Everything Together and Submitting to Kaggle

You should use your trained model to predict labels for all the news in `TestData.txt`. Output your predictions to a **csv** file and submit it to kaggle. The format should follow Part 6 as well.

Use this Kaggle [link](https://www.kaggle.com/t/e8e4f7ee507843d1902c168529d705c8) to submit your output.

In [35]:
# TODO: Code for predicting the test labels and generating the output file. Then submit the output file to Kaggle

# Work Distribution

**Please briefly describe how you divided the work.**

Your answer:


# Project Feedback [1 point]
It goes without saying that this is an unprecedented time. We on the course staff are trying our best to adapt our teaching, projects and everything else in the class to this new modality. We would immenselly appreciate it if you could provide us feedback (it's a super short form!!) on this project and **it's worth 1 point of your project grade**

Link to the feedback form: https://forms.gle/xfonP9rJk9Uk7zaSA 

We will use this feedback to improve both **upcoming projects** and projects for next year. 

Thank you so much!

# Submitting the Notebook

1. Go to File (upper left corner) -> Download .ipynb -> submit this downloaded file to cms
2. Run the first code block
3. Replace our placeholder for your correct Google Drive directory structure in the 2nd code block below. Run the code block
4. Put the name of this notebook into our placeholder in the 3rd code block. Run the code block
5. Then go to the folder icon on the very left panel, under the orange CO logo. Click on the folder and wait for a PDF version of your notebook to appear. Might take a few minutes.
6. Download the pdf version and submit to Gradescope

In [36]:
%%capture
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

In [37]:
%%capture
# the red text is a placeholder! Change it to your directory structure!
!cp 'drive/My Drive/Colab Notebooks/4740_FA20_p1_aak85_jb123.ipynb' ./ 

In [38]:
# the red text is a placeholder! Change it to the name of this notebook!
!jupyter nbconvert --to PDF "4740_FA20_p1_aak85_jb123.ipynb"

[NbConvertApp] WARNING | pattern u'4740_FA20_p1_aak85_jb123.ipynb' matched no files
This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--no-input
    Exclude input cells and output prompts from converted document. 
    This mode is ideal for generating code-free reports.
--stdout
    Write notebook output to stdout instead of files.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--inpl

# You are done! ✅